In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
from scipy import sparse
import joblib
import os
import logging

# Configure logger
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Set MLflow tracking URI
mlflow.set_tracking_uri("file:///mlruns")

# Set MLflow experiment
mlflow.set_experiment("Fraud and Credit Card Detection")

# Create directories if not exist
os.makedirs('models/fraud', exist_ok=True)
os.makedirs('models/credit', exist_ok=True)

logger.info("Directories created for saving models")

# Load processed data
fraud_data = pd.read_csv('processed_fraud_data_with_country.csv')
credit_card_data = pd.read_csv('processed_credit_card_data.csv')

# Ensure datetime columns are correctly parsed
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Feature Engineering for fraud_data
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.day
fraud_data['signup_month'] = fraud_data['signup_time'].dt.month
fraud_data['signup_year'] = fraud_data['signup_time'].dt.year

fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.day
fraud_data['purchase_month'] = fraud_data['purchase_time'].dt.month
fraud_data['purchase_year'] = fraud_data['purchase_time'].dt.year

# Drop original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])

# Identify categorical columns
categorical_columns = fraud_data.select_dtypes(include=['object']).columns.tolist()

# Frequency Encoding of categorical features
for col in categorical_columns:
    freq_encoding = fraud_data[col].value_counts().to_dict()
    fraud_data[col] = fraud_data[col].map(freq_encoding)

# Convert data types to more memory efficient types
for col in fraud_data.select_dtypes(include=['int64']).columns:
    fraud_data[col] = fraud_data[col].astype('int32')
for col in fraud_data.select_dtypes(include=['float64']).columns:
    fraud_data[col] = fraud_data[col].astype('float32')

# Feature and target separation for fraud detection data
X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']

# Feature and target separation for credit card data
X_credit = credit_card_data.drop(columns=['Class'])
y_credit = credit_card_data['Class']

# Handle missing values in features
imputer = SimpleImputer(strategy='mean')
X_fraud = imputer.fit_transform(X_fraud)
X_credit = imputer.fit_transform(X_credit)

# Convert X_fraud to a sparse matrix
X_fraud_sparse = sparse.csr_matrix(X_fraud)

# Handle missing values in target variables
y_fraud = y_fraud.fillna(y_fraud.mode()[0])
y_credit = y_credit.fillna(y_credit.mode()[0])

# Train-test split for fraud data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud_sparse, y_fraud, test_size=0.2, random_state=42)

# Train-test split for credit card data
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42)

In [41]:

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier(max_iter=1000)
}

# Function to train and evaluate models
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, dataset_name, model_name):
    with mlflow.start_run(run_name=f'{model_name} on {dataset_name}'):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Log parameters and metrics
        mlflow.log_params({
            'model': model_name,
            'dataset': dataset_name,
            'test_size': 0.2,
            'random_state': 42
        })
        mlflow.log_metric('accuracy', accuracy)

        # Save the model with mlflow
        mlflow.sklearn.log_model(model, f'{model_name}_model')

        # Form path without spaces
        dataset_path = dataset_name.lower().replace(' ', '_')
        model_path = model_name.lower().replace(' ', '_').replace(' ', '_')
        local_model_path = f"models/{dataset_path}/{model_path}_model.joblib"

        # Ensure directory exists
        os.makedirs(f"models/{dataset_path}", exist_ok=True)
        joblib.dump(model, local_model_path)
        logger.info(f"Model {model_name} for {dataset_name} saved to {local_model_path}")

        # Print and log classification report
        report = classification_report(y_test, y_pred)
        logger.info(f"Model: {model_name} on {dataset_name}")
        logger.info(f"Accuracy: {accuracy}")
        logger.info(report)
        print(f"Model: {model_name}")
        print(f"Accuracy: {accuracy}")
        print(report)
        print("="*60)

# Train and evaluate models for fraud detection data
for name, model in models.items():
    train_and_evaluate_model(model, X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud, 'Fraud Data', name)

# Train and evaluate models for credit card data
for name, model in models.items():
    train_and_evaluate_model(model, X_train_credit, y_train_credit, X_test_credit, y_test_credit, 'Credit Card Data', name)

2025/02/06 15:21:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no p

Model: Logistic Regression
Accuracy: 0.9057009562253913
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     27373
           1       0.00      0.00      0.00      2850

    accuracy                           0.91     30223
   macro avg       0.45      0.50      0.48     30223
weighted avg       0.82      0.91      0.86     30223



2025/02/06 15:21:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Decision Tree
Accuracy: 0.9112265493167455
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     27373
           1       0.53      0.59      0.56      2850

    accuracy                           0.91     30223
   macro avg       0.74      0.77      0.75     30223
weighted avg       0.92      0.91      0.91     30223



2025/02/06 15:26:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Model: Random Forest
Accuracy: 0.9995594403129736
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.99      0.73      0.84        90

    accuracy                           1.00     56746
   macro avg       0.99      0.87      0.92     56746
weighted avg       1.00      1.00      1.00     56746



2025/02/06 15:42:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: Gradient Boosting
Accuracy: 0.9992951045007578
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.89      0.63      0.74        90

    accuracy                           1.00     56746
   macro avg       0.95      0.82      0.87     56746
weighted avg       1.00      1.00      1.00     56746



2025/02/06 15:44:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model: MLP
Accuracy: 0.9982201388644133
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56656
           1       0.46      0.78      0.58        90

    accuracy                           1.00     56746
   macro avg       0.73      0.89      0.79     56746
weighted avg       1.00      1.00      1.00     56746

